In [1]:
import os
print(os.getcwd())
os.chdir('../')
print(os.getcwd())

/homes/gws/chanwkim/vit-shapley/notebooks
/homes/gws/chanwkim/vit-shapley


In [2]:
import numpy as np
import pandas as pd 
import wandb
api = wandb.Api()

In [11]:
explanation_method_main=[["attention_last", 
                          "attention_rollout"],
                         ["gradcam", 
                          "gradcamgithub",
                          "igembedding",                                                              
                          "vanillaembedding",                          
                          "sgembedding",
                          "vargradembedding",
                          "LRP"],
                         ["leaveoneoutclassifier",
                          "riseclassifier", 
                          "ours"],
                        ["random"],
                        ]
explanation_method_main_flatten=[j for i in explanation_method_main for j in i]

dataset_name="ImageNette"
explanation_method=explanation_method_main_flatten[0]
gpus_classifier=3
backbone_type="vit_small_patch16_224"

In [12]:
explanation_method

'attention_last'

In [13]:
!gpustat

l0.cs.washington.edu    Sun Jun  5 16:53:48 2022  460.39
[0] GeForce RTX 2080 Ti | 24'C,   0 % |  6988 / 11019 MB | chanwkim(6985M)
[1] GeForce RTX 2080 Ti | 24'C,   0 % |  3505 / 11019 MB | ewein(1293M) ewein(2209M)
[2] GeForce RTX 2080 Ti | 27'C,   0 % |  5988 / 11019 MB | chanwkim(5985M)
[3] GeForce RTX 2080 Ti | 24'C,   0 % |     3 / 11019 MB |
[4] GeForce RTX 2080 Ti | 26'C,   1 % |   492 / 11019 MB | chanwkim(491M)
[5] GeForce RTX 2080 Ti | 25'C,   0 % |     3 / 11019 MB |
[6] GeForce RTX 2080 Ti | 25'C,   0 % |     3 / 11019 MB |
[7] GeForce RTX 2080 Ti | 24'C,   0 % |  2462 / 11019 MB | clin25(2459M)


In [8]:
def get_retraining_status(backbone_type, dataset_name, api_dir="ch6845/transformer_interpretability_project_retraining"):
    api.flush()
    runs = api.runs(api_dir)
    result_dict_list=[]
    for run in runs:
        try:
            print(run.name)
            if run.config["datasets"]!=dataset_name:
                print(run.name, "pass")
                continue
            if run.config["backbone_type"]!=backbone_type:
                print(run.name, "pass")
                continue
            #print(run.config)
            result_dict_list.append({"name": run.name,
                                     "accuracy": run.summary["test/accuracy"] if "test/accuracy" in run.summary.keys() else np.nan,
                                     "cohenkappa": run.summary["test/cohenkappa"] if "test/cohenkappa" in run.summary.keys() else np.nan,
                                     "backbone_type": run.config["classifier_backbone_type"],
                                     "datasets": run.config["datasets"],
                                     "classifier_enable_pos_embed": run.config["classifier_enable_pos_embed"] if "classifier_enable_pos_embed" in run.config else True,

                                     "explanation_location_train": run.config["explanation_location_train"],
                                     "explanation_mask_amount_train": run.config["explanation_mask_amount_train"],
                                     "explanation_mask_ascending_train": run.config["explanation_mask_ascending_train"],

                                     "explanation_location_val": run.config["explanation_location_val"],
                                     "explanation_mask_amount_val": run.config["explanation_mask_amount_val"],
                                     "explanation_mask_ascending_val": run.config["explanation_mask_ascending_val"],

                                     "explanation_location_test": run.config["explanation_location_test"],
                                     "explanation_mask_amount_test": run.config["explanation_mask_amount_test"],
                                     "explanation_mask_ascending_test": run.config["explanation_mask_ascending_test"]

                                    })
        except:
            print(run.name, 'error')

        #print('\n')
    result_df=pd.DataFrame(result_dict_list)
    #result_df=result_df[(result_df["backbone_type"]==backbone_type) & (result_df["datasets"]==dataset_name)]
    return result_df

In [10]:
for insert_delete in ["delete", "insert"]:
    for num_stage in [1, 3, 7, 14, 28, 56, 84, 112, 140, 168, 182]:
        exp_name=f"{dataset_name}_ROAR_classifier_{backbone_type}_1e-5_train_{insert_delete}_{explanation_method}_{num_stage}"
        
        classifier_enable_pos_embed=True
        
        explanation_location_train = f"/homes/gws/chanwkim/ViT_shapley/results/4_1_explanation_generate/{dataset_name}/vit_base_patch16_224_{explanation_method}_train.pickle"
        explanation_mask_amount_train = 196-num_stage if insert_delete=="insert" else num_stage
        explanation_mask_ascending_train = True if insert_delete=="insert" else False
        
        explanation_location_val = f"/homes/gws/chanwkim/ViT_shapley/results/4_1_explanation_generate/{dataset_name}/vit_base_patch16_224_{explanation_method}_val.pickle"
        explanation_mask_amount_val = 196-num_stage if insert_delete=="insert" else num_stage
        explanation_mask_ascending_val = True if insert_delete=="insert" else False
        
        explanation_location_test = f"/homes/gws/chanwkim/ViT_shapley/results/4_1_explanation_generate/{dataset_name}/vit_base_patch16_224_{explanation_method}_test.pickle"
        explanation_mask_amount_test = 196-num_stage if insert_delete=="insert" else num_stage
        explanation_mask_ascending_test = True if insert_delete=="insert" else False
        
        retraining_status=get_retraining_status(backbone_type=backbone_type, 
                                                dataset_name=dataset_name,
                                                api_dir="ch6845/transformer_interpretability_project_retraining")
        
        if dataset_name=="ImageNette":
            checkpoint_metric="accuracy"
            loss_weight=''
        elif dataset_name=="MURA":
            checkpoint_metric="CohenKappa"
            loss_weight=" 'loss_weight = [21935, 14873]'"
        else:
            raise
        
        if len(retraining_status)==0 or not ((~retraining_status["accuracy"].isnull()) &\
                                             (retraining_status["classifier_enable_pos_embed"]==classifier_enable_pos_embed) &\
                                             (retraining_status["explanation_location_train"]==explanation_location_train) &\
                                             (retraining_status["explanation_mask_amount_train"]==explanation_mask_amount_train) &\
                                             (retraining_status["explanation_mask_ascending_train"]==explanation_mask_ascending_train) &\
                                             (retraining_status["explanation_location_val"]==explanation_location_val) &\
                                             (retraining_status["explanation_mask_amount_val"]==explanation_mask_amount_val) &\
                                             (retraining_status["explanation_mask_ascending_val"]==explanation_mask_ascending_val) &\
                                             (retraining_status["explanation_location_test"]==explanation_location_test) &\
                                             (retraining_status["explanation_mask_amount_test"]==explanation_mask_amount_test) &\
                                             (retraining_status["explanation_mask_ascending_test"]==explanation_mask_ascending_test)).any():
            command = f"~/miniconda3/envs/vitshapley/bin/python main.py with 'stage = \"classifier\"' \
            'wandb_project_name = \"transformer_interpretability_project_retraining\"' 'exp_name = \"{exp_name}\"' \
            env_chanwkim 'gpus_classifier=[{gpus_classifier}]' \
            dataset_{dataset_name}_ROAR \
            'classifier_enable_pos_embed=[{classifier_enable_pos_embed}]' \
            'explanation_location_train = \"{explanation_location_train}\"' 'explanation_mask_amount_train = {explanation_mask_amount_train}' 'explanation_mask_ascending_train = {explanation_mask_ascending_train}' \
            'explanation_location_val = \"{explanation_location_val}\"' 'explanation_mask_amount_val = {explanation_mask_amount_val}' 'explanation_mask_ascending_val = {explanation_mask_ascending_val}' \
            'explanation_location_test = \"{explanation_location_test}\"' 'explanation_mask_amount_test = {explanation_mask_amount_test}' 'explanation_mask_ascending_test = {explanation_mask_ascending_test}' \
            'classifier_backbone_type = \"{backbone_type}\"' 'classifier_download_weight = True' 'classifier_load_path = None' \
            training_hyperparameters_transformer{loss_weight} 'checkpoint_metric = \"{checkpoint_metric}\"' 'learning_rate = 1e-5'"
            print(command)
            !{command}



220604_230643_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_delete_random_1
220604_230643_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_delete_random_1 error
220602_083817_ImageNette_ROAR_classifier_vit_base_patch16_224_1e-5_train_insert_gradcamgithub_182
220602_083817_ImageNette_ROAR_classifier_vit_base_patch16_224_1e-5_train_insert_gradcamgithub_182 pass
220602_081351_ImageNette_ROAR_classifier_vit_base_patch16_224_1e-5_train_insert_gradcamgithub_168
220602_081351_ImageNette_ROAR_classifier_vit_base_patch16_224_1e-5_train_insert_gradcamgithub_168 pass
220602_074920_ImageNette_ROAR_classifier_vit_base_patch16_224_1e-5_train_insert_gradcamgithub_140
220602_074920_ImageNette_ROAR_classifier_vit_base_patch16_224_1e-5_train_insert_gradcamgithub_140 pass
220602_072449_ImageNette_ROAR_classifier_vit_base_patch16_224_1e-5_train_insert_gradcamgithub_112
220602_072449_ImageNette_ROAR_classifier_vit_base_patch16_224_1e-5_train_insert_gradcamgithub_112 pass
220602_070

220531_155334_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_insert_LRP_56
220531_155156_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_insert_vargradembedding_56
220531_154808_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_insert_riseclassifier_56
220531_154745_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_insert_ours_56
220531_154745_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_insert_leaveoneoutclassifier_56
220531_154625_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_insert_attention_rollout_56
220531_154615_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_insert_sgembedding_56
220531_154610_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_insert_gradcam_56
220531_154605_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_insert_vanillaembedding_56
220531_154604_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_insert_igembedding_56
220531_154553_ImageNette_ROAR_

220531_135017_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_delete_riseclassifier_112
220531_135017_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_delete_ours_112
220531_135017_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_delete_vargradembedding_112
220531_135017_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_delete_leaveoneoutclassifier_112
220531_135004_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_delete_sgembedding_112
220531_135003_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_delete_gradcam_112
220531_135002_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_delete_attention_rollout_112
220531_135002_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_delete_vanillaembedding_112
220531_134957_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_delete_igembedding_112
220531_134950_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_delete_attention_last_112
220531_13

220517_233223_ImageNette_ROAR_classifier_vit_base_patch16_224_1e-5_train_delete_random_28
220517_233223_ImageNette_ROAR_classifier_vit_base_patch16_224_1e-5_train_delete_random_28 pass
220517_230615_ImageNette_ROAR_classifier_vit_base_patch16_224_1e-5_train_insert_random_14
220517_230615_ImageNette_ROAR_classifier_vit_base_patch16_224_1e-5_train_insert_random_14 pass
220517_230118_ImageNette_ROAR_classifier_vit_base_patch16_224_1e-5_train_delete_random_14
220517_230118_ImageNette_ROAR_classifier_vit_base_patch16_224_1e-5_train_delete_random_14 pass
220517_223504_ImageNette_ROAR_classifier_vit_base_patch16_224_1e-5_train_insert_random_7
220517_223504_ImageNette_ROAR_classifier_vit_base_patch16_224_1e-5_train_insert_random_7 pass
220517_223113_ImageNette_ROAR_classifier_vit_base_patch16_224_1e-5_train_delete_random_7
220517_223113_ImageNette_ROAR_classifier_vit_base_patch16_224_1e-5_train_delete_random_7 pass
220517_220601_ImageNette_ROAR_classifier_vit_base_patch16_224_1e-5_train_insert

220516_092956_ImageNette_ROAR_classifier_vit_base_patch16_224_1e-5_train_insert_attention_rollout_140
220516_092956_ImageNette_ROAR_classifier_vit_base_patch16_224_1e-5_train_insert_attention_rollout_140 pass
220516_091527_ImageNette_ROAR_classifier_vit_base_patch16_224_1e-5_train_insert_vanillaembedding_140
220516_091527_ImageNette_ROAR_classifier_vit_base_patch16_224_1e-5_train_insert_vanillaembedding_140 pass
220516_091527_ImageNette_ROAR_classifier_vit_base_patch16_224_1e-5_train_insert_igembedding_140
220516_091527_ImageNette_ROAR_classifier_vit_base_patch16_224_1e-5_train_insert_igembedding_140 pass
220516_091214_ImageNette_ROAR_classifier_vit_base_patch16_224_1e-5_train_insert_leaveoneoutclassifier_56
220516_091214_ImageNette_ROAR_classifier_vit_base_patch16_224_1e-5_train_insert_leaveoneoutclassifier_56 pass
220516_090918_ImageNette_ROAR_classifier_vit_base_patch16_224_1e-5_train_insert_attention_last_140
220516_090918_ImageNette_ROAR_classifier_vit_base_patch16_224_1e-5_train_

...

220515_230230_ImageNette_ROAR_classifier_vit_base_patch16_224_1e-5_train_delete_gradcam_3
220515_230230_ImageNette_ROAR_classifier_vit_base_patch16_224_1e-5_train_delete_gradcam_3 pass
220515_230225_ImageNette_ROAR_classifier_vit_base_patch16_224_1e-5_train_delete_leaveoneoutclassifier_3
220515_230225_ImageNette_ROAR_classifier_vit_base_patch16_224_1e-5_train_delete_leaveoneoutclassifier_3 pass
220515_230056_ImageNette_ROAR_classifier_vit_base_patch16_224_1e-5_train_delete_sgembedding_3
220515_230056_ImageNette_ROAR_classifier_vit_base_patch16_224_1e-5_train_delete_sgembedding_3 pass
220515_230057_ImageNette_ROAR_classifier_vit_base_patch16_224_1e-5_train_delete_riseclassifier_3
220515_230057_ImageNette_ROAR_classifier_vit_base_patch16_224_1e-5_train_delete_riseclassifier_3 pass
220515_230028_ImageNette_ROAR_classifier_vit_base_patch16_224_1e-5_train_delete_LRP_3
220515_230028_ImageNette_ROAR_classifier_vit_base_patch16_224_1e-5_train_delete_LRP_3 pass
220515_230004_ImageNette_ROAR_cla

wandb: Currently logged in as: ch6845 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.17 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.12.10
wandb: Syncing run 220605_032056_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_insert_random_168
wandb: ⭐️ View project at https://wandb.ai/ch6845/transformer_interpretability_project_retraining
wandb: 🚀 View run at https://wandb.ai/ch6845/transformer_interpretability_project_retraining/runs/si8tlf51
wandb: Run data is saved locally in /homes/gws/chanwkim/network_drive/l0.cs.washington.edu/wandb/run-20220605_032056-si8tlf51
wandb: Run `wandb offline` to turn off syncing.

INFO - vit_shapley.modules.classifier - The backbone parameters were initialized with the downloaded pretrained weights.
Classifier(
  (backbone): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 384, kernel_size=(16, 16), stride=(16, 16))
      

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]
/homes/gws/chanwkim/miniconda3/envs/vitshapley/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(

   | Name             | Type              | Params
--------------------------------------------------------
0  | backbone         | VisionTransformer | 21.7 M
1  | head             | Linear            | 3.9 K 
2  | train_loss       | MeanMetric        | 0     
3  | train_accuracy   | Accuracy          | 0     
4  | train_precision  | Precision         | 0     
5  | train_recall     | Recall            | 0     
6  | train_f1         | F1Score           | 0     
7  | train_cohenkappa | CohenKappa        | 0     
8  | val_loss         | MeanMetric        | 0     
9  | val_accuracy     | Accuracy 

Epoch 3:  84%|█████▊ | 148/177 [00:25<00:04,  5.91it/s, loss=0.0166, v_num=lf51]
Validating: 0it [00:00, ?it/s]
Epoch 4:   5%|▎       | 8/177 [00:01<00:34,  4.93it/s, loss=0.00995, v_num=lf51]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
for insert_delete in ["delete", "insert"]:
    for num_stage in ([1, 3, 7, 14, 28, 56, 84, 112, 140, 168, 182] if insert_delete=="insert" else [1, 3, 7, 14, 28, 56, 84, 112, 140, 168, 182][::-1]):
        exp_name=f"{dataset_name}_ROAR_classifier_{backbone_type}_1e-5_train_{insert_delete}_{explanation_method}_{num_stage}_nopos"
        
        classifier_enable_pos_embed=False
        
        explanation_location_train = f"/homes/gws/chanwkim/ViT_shapley/results/4_1_explanation_generate/{dataset_name}/vit_base_patch16_224_{explanation_method}_train.pickle"
        explanation_mask_amount_train = 196-num_stage if insert_delete=="insert" else num_stage
        explanation_mask_ascending_train = True if insert_delete=="insert" else False
        
        explanation_location_val = f"/homes/gws/chanwkim/ViT_shapley/results/4_1_explanation_generate/{dataset_name}/vit_base_patch16_224_{explanation_method}_val.pickle"
        explanation_mask_amount_val = 196-num_stage if insert_delete=="insert" else num_stage
        explanation_mask_ascending_val = True if insert_delete=="insert" else False
        
        explanation_location_test = f"/homes/gws/chanwkim/ViT_shapley/results/4_1_explanation_generate/{dataset_name}/vit_base_patch16_224_{explanation_method}_test.pickle"
        explanation_mask_amount_test = 196-num_stage if insert_delete=="insert" else num_stage
        explanation_mask_ascending_test = True if insert_delete=="insert" else False
        
        retraining_status=get_retraining_status(backbone_type=backbone_type, 
                                                dataset_name=dataset_name,
                                                api_dir="ch6845/transformer_interpretability_project_retraining_nopos")
        
        if dataset_name=="ImageNette":
            checkpoint_metric="accuracy"
            loss_weight=''
        elif dataset_name=="MURA":
            checkpoint_metric="CohenKappa"
            loss_weight=" 'loss_weight = [21935, 14873]'"
        else:
            raise
        
        if len(retraining_status)==0 or not ((~retraining_status["accuracy"].isnull()) &\
                                             (retraining_status["classifier_enable_pos_embed"]==classifier_enable_pos_embed) &\
                                             (retraining_status["explanation_location_train"]==explanation_location_train) &\
                                             (retraining_status["explanation_mask_amount_train"]==explanation_mask_amount_train) &\
                                             (retraining_status["explanation_mask_ascending_train"]==explanation_mask_ascending_train) &\
                                             (retraining_status["explanation_location_val"]==explanation_location_val) &\
                                             (retraining_status["explanation_mask_amount_val"]==explanation_mask_amount_val) &\
                                             (retraining_status["explanation_mask_ascending_val"]==explanation_mask_ascending_val) &\
                                             (retraining_status["explanation_location_test"]==explanation_location_test) &\
                                             (retraining_status["explanation_mask_amount_test"]==explanation_mask_amount_test) &\
                                             (retraining_status["explanation_mask_ascending_test"]==explanation_mask_ascending_test)).any():
            command = f"~/miniconda3/envs/vitshapley/bin/python main.py with 'stage = \"classifier\"' \
            'wandb_project_name = \"transformer_interpretability_project_retraining_nopos\"' 'exp_name = \"{exp_name}\"' \
            env_chanwkim 'gpus_classifier=[{gpus_classifier}]' \
            dataset_{dataset_name}_ROAR \
            'classifier_enable_pos_embed={classifier_enable_pos_embed}' \
            'explanation_location_train = \"{explanation_location_train}\"' 'explanation_mask_amount_train = {explanation_mask_amount_train}' 'explanation_mask_ascending_train = {explanation_mask_ascending_train}' \
            'explanation_location_val = \"{explanation_location_val}\"' 'explanation_mask_amount_val = {explanation_mask_amount_val}' 'explanation_mask_ascending_val = {explanation_mask_ascending_val}' \
            'explanation_location_test = \"{explanation_location_test}\"' 'explanation_mask_amount_test = {explanation_mask_amount_test}' 'explanation_mask_ascending_test = {explanation_mask_ascending_test}' \
            'classifier_backbone_type = \"{backbone_type}\"' 'classifier_download_weight = True' 'classifier_load_path = None' \
            training_hyperparameters_transformer{loss_weight} 'checkpoint_metric = \"{checkpoint_metric}\"' 'learning_rate = 1e-5'"
            print(command)
            !{command}



In [15]:
for insert_delete in ["insert"]:
    for num_stage in ([0]):
        exp_name=f"{dataset_name}_ROAR_classifier_{backbone_type}_1e-5_train_{insert_delete}_{explanation_method}_{num_stage}_nopos"
        
        classifier_enable_pos_embed=False
        
        explanation_location_train = f"/homes/gws/chanwkim/ViT_shapley/results/4_1_explanation_generate/{dataset_name}/vit_base_patch16_224_{explanation_method}_train.pickle"
        explanation_mask_amount_train = 196-num_stage if insert_delete=="insert" else num_stage
        explanation_mask_ascending_train = True if insert_delete=="insert" else False
        
        explanation_location_val = f"/homes/gws/chanwkim/ViT_shapley/results/4_1_explanation_generate/{dataset_name}/vit_base_patch16_224_{explanation_method}_val.pickle"
        explanation_mask_amount_val = 196-num_stage if insert_delete=="insert" else num_stage
        explanation_mask_ascending_val = True if insert_delete=="insert" else False
        
        explanation_location_test = f"/homes/gws/chanwkim/ViT_shapley/results/4_1_explanation_generate/{dataset_name}/vit_base_patch16_224_{explanation_method}_test.pickle"
        explanation_mask_amount_test = 196-num_stage if insert_delete=="insert" else num_stage
        explanation_mask_ascending_test = True if insert_delete=="insert" else False
        
        retraining_status=get_retraining_status(backbone_type=backbone_type, 
                                                dataset_name=dataset_name,
                                                api_dir="ch6845/transformer_interpretability_project_retraining_nopos")
        
        if dataset_name=="ImageNette":
            checkpoint_metric="accuracy"
            loss_weight=''
        elif dataset_name=="MURA":
            checkpoint_metric="CohenKappa"
            loss_weight=" 'loss_weight = [21935, 14873]'"
        else:
            raise
        
        if len(retraining_status)==0 or not ((~retraining_status["accuracy"].isnull()) &\
                                             (retraining_status["classifier_enable_pos_embed"]==classifier_enable_pos_embed) &\
                                             (retraining_status["explanation_location_train"]==explanation_location_train) &\
                                             (retraining_status["explanation_mask_amount_train"]==explanation_mask_amount_train) &\
                                             (retraining_status["explanation_mask_ascending_train"]==explanation_mask_ascending_train) &\
                                             (retraining_status["explanation_location_val"]==explanation_location_val) &\
                                             (retraining_status["explanation_mask_amount_val"]==explanation_mask_amount_val) &\
                                             (retraining_status["explanation_mask_ascending_val"]==explanation_mask_ascending_val) &\
                                             (retraining_status["explanation_location_test"]==explanation_location_test) &\
                                             (retraining_status["explanation_mask_amount_test"]==explanation_mask_amount_test) &\
                                             (retraining_status["explanation_mask_ascending_test"]==explanation_mask_ascending_test)).any():
            command = f"~/miniconda3/envs/vitshapley/bin/python main.py with 'stage = \"classifier\"' \
            'wandb_project_name = \"transformer_interpretability_project_retraining_nopos\"' 'exp_name = \"{exp_name}\"' \
            env_chanwkim 'gpus_classifier=[{gpus_classifier}]' \
            dataset_{dataset_name}_ROAR \
            'classifier_enable_pos_embed={classifier_enable_pos_embed}' \
            'explanation_location_train = \"{explanation_location_train}\"' 'explanation_mask_amount_train = {explanation_mask_amount_train}' 'explanation_mask_ascending_train = {explanation_mask_ascending_train}' \
            'explanation_location_val = \"{explanation_location_val}\"' 'explanation_mask_amount_val = {explanation_mask_amount_val}' 'explanation_mask_ascending_val = {explanation_mask_ascending_val}' \
            'explanation_location_test = \"{explanation_location_test}\"' 'explanation_mask_amount_test = {explanation_mask_amount_test}' 'explanation_mask_ascending_test = {explanation_mask_ascending_test}' \
            'classifier_backbone_type = \"{backbone_type}\"' 'classifier_download_weight = True' 'classifier_load_path = None' \
            training_hyperparameters_transformer{loss_weight} 'checkpoint_metric = \"{checkpoint_metric}\"' 'learning_rate = 1e-5'"
            print(command)
            !{command}



220605_165339_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_delete_random_0_nopos
220605_165339_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_delete_random_0_nopos pass
220605_033323_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_insert_random_182_nopos
220605_032056_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_insert_random_168_nopos
220605_030826_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_insert_random_140_nopos
220605_025603_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_insert_random_112_nopos
220605_024336_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_insert_random_84_nopos
220605_023113_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_insert_random_56_nopos
220605_021851_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_insert_random_28_nopos
220605_020625_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_insert_random_14_nopos
220605_015347_ImageNett

220531_210620_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_insert_attention_last_140_nopos
220531_210411_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_insert_vargradembedding_112_nopos
220531_210324_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_insert_LRP_84_nopos
220531_205926_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_insert_riseclassifier_112_nopos
220531_205914_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_insert_ours_112_nopos
220531_205913_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_insert_leaveoneoutclassifier_112_nopos
220531_205330_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_insert_gradcam_112_nopos
220531_205325_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_insert_igembedding_112_nopos
220531_205326_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_insert_attention_rollout_112_nopos
220531_205324_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-

220531_190711_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_delete_vargradembedding_3_nopos
220531_190259_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_delete_ours_3_nopos
220531_190243_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_delete_riseclassifier_3_nopos
220531_190220_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_delete_leaveoneoutclassifier_3_nopos
220531_190156_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_delete_LRP_7_nopos
220531_185730_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_delete_attention_rollout_3_nopos
220531_185724_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_delete_vanillaembedding_3_nopos
220531_185719_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_delete_igembedding_3_nopos
220531_185714_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_delete_sgembedding_3_nopos
220531_185701_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_del

220531_170615_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_delete_ours_182_nopos
220531_170608_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_delete_riseclassifier_182_nopos
220531_170600_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_delete_leaveoneoutclassifier_182_nopos
220531_170252_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_delete_sgembedding_182_nopos
220531_170248_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_delete_gradcam_182_nopos
220531_170244_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_delete_vanillaembedding_182_nopos
220531_170235_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_delete_attention_rollout_182_nopos
220531_170230_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_delete_igembedding_182_nopos
220531_170218_ImageNette_ROAR_classifier_vit_small_patch16_224_1e-5_train_delete_attention_last_182_nopos
220523_144108_ImageNette_ROAR_classifier_vit_base_patch16

220517_025714_ImageNette_ROAR_classifier_vit_base_patch16_224_1e-5_train_insert_ours_140_nopos
220517_025714_ImageNette_ROAR_classifier_vit_base_patch16_224_1e-5_train_insert_ours_140_nopos pass
220517_025556_ImageNette_ROAR_classifier_vit_base_patch16_224_1e-5_train_insert_vargradembedding_112_nopos
220517_025556_ImageNette_ROAR_classifier_vit_base_patch16_224_1e-5_train_insert_vargradembedding_112_nopos pass
220517_024851_ImageNette_ROAR_classifier_vit_base_patch16_224_1e-5_train_insert_leaveoneoutclassifier_112_nopos
220517_024851_ImageNette_ROAR_classifier_vit_base_patch16_224_1e-5_train_insert_leaveoneoutclassifier_112_nopos pass
220517_024056_ImageNette_ROAR_classifier_vit_base_patch16_224_1e-5_train_insert_riseclassifier_112_nopos
220517_024056_ImageNette_ROAR_classifier_vit_base_patch16_224_1e-5_train_insert_riseclassifier_112_nopos pass
220517_024056_ImageNette_ROAR_classifier_vit_base_patch16_224_1e-5_train_insert_sgembedding_112_nopos
220517_024056_ImageNette_ROAR_classifier

...

Epoch 3:  84%|████████▎ | 148/177 [00:24<00:04,  5.95it/s, loss=2.3, v_num=kwbx]
Validating: 0it [00:00, ?it/s]
Epoch 4:  84%|███████▌ | 148/177 [00:24<00:04,  5.94it/s, loss=2.31, v_num=kwbx]
Validating: 0it [00:00, ?it/s]
Epoch 5:  84%|███████▌ | 148/177 [00:24<00:04,  5.92it/s, loss=2.31, v_num=kwbx]
Validating: 0it [00:00, ?it/s]
Epoch 6:  84%|████████▎ | 148/177 [00:25<00:04,  5.91it/s, loss=2.3, v_num=kwbx]
Validating: 0it [00:00, ?it/s]
Epoch 7:  84%|████████▎ | 148/177 [00:25<00:04,  5.92it/s, loss=2.3, v_num=kwbx]
Validating: 0it [00:00, ?it/s]
Epoch 7:  98%|█████████▊| 174/177 [00:26<00:00,  6.45it/s, loss=2.3, v_num=kwbx]


Epoch 8:  84%|███████▌ | 148/177 [00:24<00:04,  5.92it/s, loss=2.31, v_num=kwbx]
Validating: 0it [00:00, ?it/s]
Epoch 9:  84%|████████▎ | 148/177 [00:24<00:04,  5.93it/s, loss=2.3, v_num=kwbx]
Validating: 0it [00:00, ?it/s]
Epoch 10:  84%|██████▋ | 148/177 [00:24<00:04,  5.93it/s, loss=2.31, v_num=kwbx]
Validating: 0it [00:00, ?it/s]
Epoch 11:  84%|██████▋ | 148/177 [00:24<00:04,  5.93it/s, loss=2.31, v_num=kwbx]
Validating: 0it [00:00, ?it/s]
Epoch 12:  84%|███████▌ | 148/177 [00:24<00:04,  5.93it/s, loss=2.3, v_num=kwbx]
Validating: 0it [00:00, ?it/s]
Epoch 12:  97%|████████▋| 172/177 [00:26<00:00,  6.43it/s, loss=2.3, v_num=kwbx]


Epoch 13:  84%|███████▌ | 148/177 [00:24<00:04,  5.92it/s, loss=2.3, v_num=kwbx]
Validating: 0it [00:00, ?it/s]
Epoch 14:  84%|███████▌ | 148/177 [00:24<00:04,  5.93it/s, loss=2.3, v_num=kwbx]
Validating: 0it [00:00, ?it/s]
Epoch 15:  84%|███████▌ | 148/177 [00:25<00:04,  5.92it/s, loss=2.3, v_num=kwbx]
Validating: 0it [00:00, ?it/s]
Epoch 16:  84%|███████▌ | 148/177 [00:24<00:04,  5.93it/s, loss=2.3, v_num=kwbx]
Validating: 0it [00:00, ?it/s]
Epoch 17:  84%|██████▋ | 148/177 [00:25<00:04,  5.92it/s, loss=2.31, v_num=kwbx]
Validating: 0it [00:00, ?it/s]
Epoch 17:  98%|███████▊| 174/177 [00:26<00:00,  6.45it/s, loss=2.31, v_num=kwbx]


Epoch 18:  84%|███████▌ | 148/177 [00:25<00:04,  5.91it/s, loss=2.3, v_num=kwbx]
Validating: 0it [00:00, ?it/s]
Epoch 19:  84%|███████▌ | 148/177 [00:25<00:04,  5.90it/s, loss=2.3, v_num=kwbx]
Validating: 0it [00:00, ?it/s]
Epoch 20:  84%|███████▌ | 148/177 [00:24<00:04,  5.92it/s, loss=2.3, v_num=kwbx]
Validating: 0it [00:00, ?it/s]
Epoch 21:  84%|███████▌ | 148/177 [00:24<00:04,  5.94it/s, loss=2.3, v_num=kwbx]
Validating: 0it [00:00, ?it/s]
Epoch 22:  84%|███████▌ | 148/177 [00:24<00:04,  5.93it/s, loss=2.3, v_num=kwbx]
Validating: 0it [00:00, ?it/s]
Epoch 22:  99%|████████▉| 176/177 [00:27<00:00,  6.51it/s, loss=2.3, v_num=kwbx]


Epoch 23:  84%|███████▌ | 148/177 [00:24<00:04,  5.93it/s, loss=2.3, v_num=kwbx]
Validating: 0it [00:00, ?it/s]
Epoch 24:  84%|███████▌ | 148/177 [00:24<00:04,  5.93it/s, loss=2.3, v_num=kwbx]
Validating: 0it [00:00, ?it/s]
Epoch 24: 100%|█████████| 177/177 [00:27<00:00,  6.51it/s, loss=2.3, v_num=kwbx]Epoch 24, global step 3674: val/checkpoint_metric was not in top 1
Saving latest checkpoint...
Epoch 24: 100%|█████████| 177/177 [00:27<00:00,  6.37it/s, loss=2.3, v_num=kwbx]
INFO - vit_shapley.modules.classifier - The backbone parameters were initialized with the downloaded pretrained weights.
Global seed set to 0
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]
Testing:  97%|████████████████████████████████▉ | 30/31 [00:03<00:00,  8.83it/s]/homes/gws/chanwkim/miniconda3/envs/vitshapley/lib/python3.8/site-packages/pytorch_lightning/utilities/data.py:59: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 43. To avoid any miscalc

~/miniconda3/envs/vitshapley/bin/python main.py with 'stage = "classifier"'             'wandb_project_name = "transformer_interpretability_project_retraining"' 'exp_name = "MURA_ROAR_classifier_vit_base_patch16_224_1e-5_train_original"'             env_chanwkim 'gpus_classifier=[7]'             dataset_MURA_ROAR             'explanation_location_train = None' 'explanation_mask_amount_train = None' 'explanation_mask_ascending_train = None'             'explanation_location_val = None' 'explanation_mask_amount_val = None' 'explanation_mask_ascending_val = None'             'explanation_location_test = None' 'explanation_mask_amount_test = None' 'explanation_mask_ascending_test = None'             'classifier_backbone_type = "vit_base_patch16_224"' 'classifier_download_weight = True' 'classifier_load_path = None'             training_hyperparameters_transformer 'loss_weight = [21935, 14873]' 'checkpoint_metric = "CohenKappa"' 'learning_rate = 1e-5'

In [ ]:
~/miniconda3/envs/vitshapley/bin/python main.py with 'stage = "classifier"'             'wandb_project_name = "transformer_interpretability_project_retraining_nopos"' 'exp_name = "ImageNette_ROAR_classifier_vit_base_patch16_224_1e-5_train_insert_attention_last_0_nopos"'             env_chanwkim 'gpus_classifier=[3]'             dataset_ImageNette_ROAR             'classifier_enable_pos_embed=False'             'explanation_location_train = "/homes/gws/chanwkim/ViT_shapley/results/4_1_explanation_generate/ImageNette/vit_base_patch16_224_attention_last_train.pickle"' 'explanation_mask_amount_train = 196' 'explanation_mask_ascending_train = True'             'explanation_location_val = "/homes/gws/chanwkim/ViT_shapley/results/4_1_explanation_generate/ImageNette/vit_base_patch16_224_attention_last_val.pickle"' 'explanation_mask_amount_val = 196' 'explanation_mask_ascending_val = True'             'explanation_location_test = "/homes/gws/chanwkim/ViT_shapley/results/4_1_explanation_generate/ImageNette/vit_base_patch16_224_attention_last_test.pickle"' 'explanation_mask_amount_test = 196' 'explanation_mask_ascending_test = True'             'classifier_backbone_type = "vit_base_patch16_224"' 'classifier_download_weight = True' 'classifier_load_path = None'             training_hyperparameters_transformer 'checkpoint_metric = "accuracy"' 'learning_rate = 1e-5'